In [ ]:
!pip install -q efficientnet >> /dev/null

In [ ]:
!conda install gdcm -c conda-forge -y

In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pandas as pd


In [ ]:
import numpy as np 
import pandas as pd 
import os, shutil
from glob import glob
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import random
tqdm.pandas()

In [ ]:
image_level = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")  
study_level = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

study_level['id'] = study_level['id'].str.replace('_study','')
study_level.rename(columns={"id":"StudyID"},inplace=True)
image_level.rename(columns={"StudyInstanceUID":"StudyID"},inplace=True)
merged_df = pd.merge(image_level,study_level,on=['StudyID'])
merged_df['id'] = merged_df['id'].str.replace('_image','')
merged_df.rename(columns={"id":"ImageID"},inplace=True)

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
for split in ['test', 'train']:
    save_dir = f'/kaggle/tmp/{split}/'

    os.makedirs(save_dir, exist_ok=True)
    
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=2048)  
            im.save(os.path.join(save_dir, file.replace('dcm', 'jpg')))
            merged_df.loc[merged_df.index[merged_df['ImageID']==file],"ImagePath"] = os.path.join(save_dir, file.replace('dcm', 'jpg'))
            merged_df.loc[merged_df.index[merged_df['ImageID']==file],"Split"] = split

In [ ]:
merged_df.to_csv("/kaggle/working/merged.csv")

In [ ]:
%%time
!tar -zcf dataset.tar.gz -C "/kaggle/tmp/" .